In [1]:
import numpy as np
import pandas as pd
import os
!pip install yahooquery
import yahooquery
from pandas_datareader import data
from datetime import timedelta, date, datetime

     |████████████████████████████████| 51kB 3.5MB/s 
     |████████████████████████████████| 5.5MB 11.5MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 
  Created wheel for requests-futures: filename=requests_futures-1.0.0-cp37-none-any.whl size=7012 sha256=8e5b398ff0a8563f25fc1a3f2db9a79adf21709dee7143fd0489e01a586d5621
  Stored in directory: /root/.cache/pip/wheels/26/d0/f5/dc4e4a37bbe55c9acf967d2bd899152412c1e49c227f5395ff
Successfully built requests-futures
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [2]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn import metrics
# import seaborn as sn
# import matplotlib.pyplot as plt
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import LabelEncoder
# from sklearn.neural_network import MLPRegressor
# from sklearn.metrics import r2_score

# import matplotlib.pyplot as plt

# Get yahoo ground truth stock data

In [3]:
def get_yahoo_dataframe(symbol):
  # Get historical data
  #symbol = 'AAPL'
  ticker = yahooquery.Ticker(symbol)
  # df = pd.DataFrame(ticker.history(start='2020-06-01', end='2021-01-29'))
  avg_df=data.DataReader(symbol, 'yahoo', '20190213', '20210213').reset_index()

  # # compute weekly average and monthly average
  # avg_df['Date'] = avg_df['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
  avg_df = avg_df.set_index('Date')
  dates = pd.date_range(avg_df.index[0], avg_df.index[-1], freq='D')

  avg_df = avg_df.reindex(dates) 
  avg_df.reindex(dates)

  return avg_df

def prepare_data_Y_for_ML(df, how_long=7):
  # how_long = "weekly": 7, "monthly":40, "Quarterly":90
  advance = df['Adj Close'].shift(-how_long)
  Y = ((advance - df['Adj Close'])/df['Adj Close'])*100
  #Y=Y[how_long:]
  return Y 


In [4]:
from google.colab import drive
# drive.flush_and_unmount()
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


# One time write to csv

In [5]:
# symbol='AAPL'
# symlist = ['AAPL', 'ADI', 'AMT', 'AMZN', 'BABA', 'COUP', 'CRM', 'ERIC', 'GOOGL', 'MELI', 'MSFT', 'NVDA', 'OKTA', 'PANW', 'PYPL', 'QCOM', 'SHOP', 'SQ', 'TMUS']
# for symbol in symlist:
#   df_x=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/reddit_avg_data/{}_avg.csv'.format(symbol), index_col=0)
#   df=get_yahoo_dataframe(symbol)
#   df["short-term stock"]=prepare_data_Y_for_ML(df, how_long=7)
#   df["mid-term stock"]=prepare_data_Y_for_ML(df, how_long=40)
#   df["long-term stock"]=prepare_data_Y_for_ML(df,how_long=90)
#   df.to_csv('/content/gdrive/My Drive/Colab Notebooks/yahoo_stock_ground_truth/{}_avg.csv'.format(symbol))

# real time analysis

In [12]:
symlist = ['AAPL', 'ADI', 'AMT', 'AMZN', 'BABA', 'COUP', 'CRM', 'ERIC', 'GOOGL', 'MELI', 'MSFT', 'NVDA', 'OKTA', 'PANW', 'PYPL', 'QCOM', 'SHOP', 'SQ', 'TMUS']

all_df_list=[]
for symbol in symlist:
  df=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/yahoo_stock_ground_truth/{}_avg.csv'.format(symbol))
  all_df_list.append(df)
all_df=pd.concat(all_df_list)
all_df

,Unnamed: 0,High,Low,Open,Close,Volume,Adj Close,short-term stock,mid-term stock,long-term stock
0,2019-02-13,43.119999,42.480000,42.847500,42.544998,89960800.0,41.672295,1.087097,10.906092,11.286012
1,2019-02-14,42.814999,42.345001,42.427502,42.700001,87342800.0,41.824120,0.152217,9.361824,12.210307
2,2019-02-15,42.924999,42.437500,42.812500,42.605000,98507200.0,41.731060,1.496333,10.591499,11.965742
3,2019-02-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-02-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
726,2021-02-08,126.260002,122.540001,126.199997,126.099998,5742600.0,126.099998,NaN,NaN,NaN
727,2021-02-09,125.989998,124.050003,125.919998,124.449997,3342900.0,124.449997,NaN,NaN,NaN
728,2021-02-10,125.660004,124.320000,125.110001,124.489998,2880000.0,124.489998,NaN,NaN,NaN
729,2021-02-11,125.449997,123.459999,125.430000,124.400002,3084900.0,124.400002,NaN,NaN,NaN


In [27]:
# Twitter data
all_X_list=[]
for symbol in symlist:
  df=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/twitter_avgData/{}_Twitavg.csv'.format(symbol), index_col=0)
  all_X_list.append(df)
  
  print(df.shape)
  print(symbol+ ' start date:' +df.index[0] + ' End date:' +df.index[-1])

all_X=pd.concat(all_X_list)
all_X


(732, 5)
AAPL start date:2019-02-13 End date:2021-02-13
(732, 5)
ADI start date:2019-02-13 End date:2021-02-13
(732, 5)
AMT start date:2019-02-13 End date:2021-02-13
(732, 5)
AMZN start date:2019-02-13 End date:2021-02-13
(732, 5)
BABA start date:2019-02-13 End date:2021-02-13
(732, 5)
COUP start date:2019-02-13 End date:2021-02-13
(732, 5)
CRM start date:2019-02-13 End date:2021-02-13
(732, 5)
ERIC start date:2019-02-13 End date:2021-02-13
(732, 5)
GOOGL start date:2019-02-13 End date:2021-02-13
(732, 5)
MELI start date:2019-02-13 End date:2021-02-13
(732, 5)
MSFT start date:2019-02-13 End date:2021-02-13
(732, 5)
NVDA start date:2019-02-13 End date:2021-02-13
(732, 5)
OKTA start date:2019-02-13 End date:2021-02-13
(732, 5)
PANW start date:2019-02-13 End date:2021-02-13
(732, 5)
PYPL start date:2019-02-13 End date:2021-02-13
(732, 5)
QCOM start date:2019-02-13 End date:2021-02-13
(732, 5)
SHOP start date:2019-02-13 End date:2021-02-13
(732, 5)
SQ start date:2019-02-13 End date:2021-02

,Daily average,weekly average,monthly average,long term average,short term average
2019-02-13,0.192797,NaN,NaN,NaN,NaN
2019-02-14,0.325841,NaN,NaN,NaN,NaN
2019-02-15,0.126100,NaN,NaN,NaN,NaN
2019-02-16,0.206790,NaN,NaN,NaN,NaN
2019-02-17,0.235917,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2021-02-09,0.088400,0.350547,1.204325,4.823876,0.350547
2021-02-10,0.043909,0.341746,1.214762,4.726012,0.341746
2021-02-11,0.047590,0.321151,1.225261,4.633190,0.321151
2021-02-12,0.041406,0.288077,1.235959,4.544684,0.288077
